In [89]:
import numpy as np
import pandas as pd 
import ast


In [90]:
movies= pd.read_csv(r'C:\Users\madha\OneDrive\Desktop\movie recommender system\tmdb_5000_movies.csv')
credit= pd.read_csv(r'C:\Users\madha\OneDrive\Desktop\movie recommender system\tmdb_5000_credits.csv')


In [91]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [92]:
movies=movies.merge(credit,on='title')

genres,keywords,id,language(for this not needed as majority is english),title,overview,popularity(if needed),cast,crew

In [93]:
movies = movies [['id','title','overview','genres','keywords','cast','crew']] 

In [94]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [95]:
movies.isnull().sum()
#shows number of entries for each colums that are empty
    #originally had 3 for overview but after running the next code becomes 0

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [96]:
movies.dropna(inplace=True)
#this removes the entries that have are empty (null) from the dataframe movies

In [97]:
movies.duplicated().sum()
#shows number of duplicated rows in the dataframe

0

In [98]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

-the genres here are a list of dictionaries with id name and the genre names
-creating a helper function to concatenate only the genres into a list instead of a list of dictionaries as given in the dataset,
    ie making a list of the 3rd elements of each dictionary

In [99]:
#here the first element is a string,which doesnt work,so we use the library ast to literal evaluate the first element of each dictionary and concat
def concat(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [100]:
movies['genres']=movies['genres'].apply(concat)
#this applies the concat function to specified columns and reassigns that colums to the updated one

In [101]:
movies['keywords'] = movies['keywords'].apply(concat)

In [102]:
#now we create a function to return the top 3 actors for each movie with a new concat function
def concatcast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1 
        else:
            break
    return L

In [103]:
movies['cast'] = movies['cast'].apply(concatcast)

In [104]:
#create a function to extract only the director for eacgh movie and put that as the crew
def directorcall(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=="Director":
            L.append(i['name'])
            break
    return L

In [105]:
movies['crew'] = movies['crew'].apply(directorcall)

In [106]:
#removing spaces from each entry for all columns so the model does not get confused between first names

movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [107]:
#making a new column that is a combination of all the columns we hav right now as a simple tags column
movies['tags']=movies['genres'] + movies['keywords'] + movies['cast']+movies['crew']

In [108]:
#making a new datafram with just the id name and tags
new_df=movies[['id','title','tags']]
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
#this remove commas and makes it all one string with words seperated by spaces ^^
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())



C:\Users\madha\AppData\Local\Temp\ipykernel_18352\547062105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
C:\Users\madha\AppData\Local\Temp\ipykernel_18352\547062105.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


 use the bag of words tech which is where we concat all the tags and sort the words by frequency,and make a vector for each movie for frequency of each of those words in its tag
 now we find closest vectors based on this vector in a 5000(number of most common words depends on set) dimensional space
 have to disclude stop words like are the and a and such

 we use the sckit learn count vectorizer which does this automatically

In [118]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [119]:
vectors = cv.fit_transform(new_df['tags']).toarray()
#this performs the bag of words using sklearn and send each vector to an array from the tags of each movie(all automatic with the library) and sets it to the vectors list

In [121]:
cv.get_feature_names_out()
#just see if the stemming was successful

array(['16thcenturi', '17thcenturi', '18thcenturi', ..., 'zooeydeschanel',
       'zookeep', 'zoëkravitz'], dtype=object)

in the top 5000 words we have,there are words that are repeated as different tenses or versions,such as action and actions,love loving,etc

here we use the nltk library(natural language) to help remove those cases with something called stemming

In [160]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
import difflib

In [171]:
#function to stem every word in the tag with nltk
def stemming(text):
    y=[ ]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [172]:
new_df['tags']=new_df['tags'].apply(stemming)

C:\Users\madha\AppData\Local\Temp\ipykernel_18352\226593929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stemming)


now to find the distance between vectors,instead of finding the direct distance as in the square sum over square root,we'll find the angle as it is computationally cheaper and more accurate when dealing with higher dimensional vectors

In [122]:
# we use the sklearn cosine similarity function to do this
from sklearn.metrics.pairwise import cosine_similarity


In [127]:
similarity=cosine_similarity(vectors)
similarity
#this gives us the similarity between each movie so from here,intuitively we can just sort the similarities and give the top 5 not including the 1(which is the movie comparing it to itself))

array([[1.        , 0.12309149, 0.11605177, ..., 0.06085806, 0.        ,
        0.        ],
       [0.12309149, 1.        , 0.12856487, ..., 0.        , 0.        ,
        0.        ],
       [0.11605177, 0.12856487, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06085806, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [187]:

# here the input is the name of the movie so we have to find the index to easilyt find the similar movies in the similarity list
import difflib
def recommend(movie):
    
    closest=difflib.get_close_matches(movie,new_df['title'],n=1)
    movie=closest[0]
    movie_index=new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    #when we sort the distances,we lose the index of each movie,so to fix this,we use the enumerate function which makes the list a list of tuples,which pairs each movie with its original index
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        #print(i[0]) # this would be the id of the movie
        print(new_df.iloc[i[0]].title)
    #print(closest)
    

thats it now we can use the recommend function to give the top 5 closest movies to a given input! #modified code so that it takes the movie name closest to input(doesnt have to be exact movie name now)

In [191]:
recommend('avengers 2')

Avengers: Age of Ultron
Captain America: Civil War
Captain America: The Winter Soldier
Ant-Man
Iron Man 2
